# Validation Study 

we have 8 sensors 0-7, but sensor 1 is not connected, ie to tal of seven sensor data with sensor 1 being dead

In [1]:
%matplotlib inline


In [2]:
import os
import pandas as pd
from pandas import DataFrame
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
#import seaborn as sns

In [3]:
# reading data into dataframes 

path = os.getcwd()+"\RM_3_20191101\WAVE"

REST_DF_0 = pd.read_csv(path + "\RM_3_20191101\RM_3_20191101_001_REST.tsv", delimiter='\t', skiprows=1,header=None)
REST_DF_1 = pd.read_csv(path + "\RM_3_20191101\RM_3_20191101_002_REST_1.tsv", delimiter='\t', skiprows=1,header=None)

OPEN_DF_HOL_0 = pd.read_csv(path +"\RM_3_20191101\RM_3_20191101_003_OPEN_HOLD.tsv", delimiter='\t', skiprows=1,header=None)
OPEN_DF_HOL_1 = pd.read_csv(path +"\RM_3_20191101\RM_3_20191101_004_OPEN_HOLD_1.tsv", delimiter='\t', skiprows=1,header=None)
#BIGSMILE_DF = pd.read_csv('RM_1_20190913_004_BIGSMILE.tsv', delimiter='\t', skiprows=1,header=None)

In [4]:
def cleaning_df(DF):
    # dropping columns with no value
    #DF = DF.dropna(axis =1) 
    # dropping wave id column
    DF = DF.drop(columns = 2, axis =1)
    # name the first sensor columns 
    cols_0={0: "Time", 1: "FrameID", 3: "SensorID", 4: "Sensor_1Status", 5: "X_Nose", 6: "Y_Nose", 7:"Z_Nose"}
    DF = DF.rename(columns=cols_0, errors="raise")
    DF = DF.drop(columns = [8,9,10, 11], axis =1)
    
    DF = DF.drop(columns = [12,13,14,15,16,17,18,19,20], axis =1)
    cols_1={21: "SensorID", 22: "Sensor_2Status", 23: "X_LLeft", 24: "Y_LLeft", 25:"Z_LLeft"}
    DF = DF.rename(columns=cols_1, errors="raise")
    
    DF = DF.drop(columns = [26,27,28,29], axis =1)
    cols_2={30: "SensorID", 31: "Sensor_3Status", 32: "X_LR", 33: "Y_LR", 34:"Z_LR"}
    DF = DF.rename(columns=cols_2, errors="raise")
    
    DF = DF.drop(columns = [35,36,37,38], axis =1)
    cols_3={39: "SensorID", 40: "Sensor_4Status", 41: "X_UL", 42: "Y_UL", 43:"Z_UL"}
    DF = DF.rename(columns=cols_3, errors="raise")
    
    DF = DF.drop(columns = [44,44,46,47], axis =1)
    cols_4={48: "SensorID", 49: "Sensor_5Status", 50: "X_LL", 51: "Y_LL", 52:"Z_LL"}
    DF = DF.rename(columns=cols_4, errors="raise")
    
    DF = DF.drop(columns = [53,54,55,56], axis =1)
    cols_5={57: "SensorID", 58: "Sensor_6Status", 59: "X_JR", 60: "Y_JR", 61:"Z_JR"}
    DF = DF.rename(columns=cols_5, errors="raise")
    
    DF = DF.drop(columns = [62,63,64,65], axis =1)
    cols_6={66: "SensorID", 67: "Senson_7Status", 68: "X_JL", 69: "Y_JL", 70:"Z_JL"}
    DF = DF.rename(columns=cols_6, errors="raise")
    DF = DF.drop(columns = [71,72,73,74], axis =1)
    return DF

In [5]:
REST_DF_0 = cleaning_df(REST_DF_0)
REST_DF_1 = cleaning_df(REST_DF_1)

OPEN_DF_HOL_0 = cleaning_df(OPEN_DF_HOL_0)
OPEN_DF_HOL_1 = cleaning_df(OPEN_DF_HOL_1)

#BIGSMILE_DF = cleaning_df(BIGSMILE_DF)

In [6]:
def LipEdgeDist(DF):
    DF['Horiz_Lip_Motion'] = DF.apply(lambda row: math.sqrt((row.X_LLeft - row.X_LR)**2 + (row.Y_LLeft - row.Y_LR)**2
                                                                  + (row.Z_LLeft - row.Z_LR)**2), axis = 1) 
    DF['Vert_Lip_Motion'] = DF.apply(lambda row: math.sqrt((row.X_UL - row.X_LL)**2 + (row.Y_UL - row.Y_LL)**2 +
                                                                 (row.Z_UL - row.Z_LL)**2), axis = 1) 
    return DF

In [7]:
REST_DF_0 = LipEdgeDist(REST_DF_0)
REST_DF_1 = LipEdgeDist(REST_DF_1)

OPEN_DF_HOL_0 = LipEdgeDist(OPEN_DF_HOL_0)
OPEN_DF_HOL_1 = LipEdgeDist(OPEN_DF_HOL_1)

#BIGSMILE_DF = LipEdgeDist(BIGSMILE_DF)

In [8]:
header = ["Time", 'Horiz_Lip_Motion', 'Vert_Lip_Motion']

OPEN_DF.to_csv('open_sensor_results.csv', columns = header)

BIGSMILE_DF.to_csv('bigsmile_sensor_results.csv', columns = header)

REST_DF.to_csv('rest_sensor_results.csv', columns = header)

NameError: name 'OPEN_DF' is not defined

In [ ]:
REST_DF.plot(x='FrameID', y='Horiz_Lip_Motion', style='o')

In [ ]:
REST_DF.plot(x='Time', y='Vert_Lip_Motion', style='o')

In [ ]:
BIGSMILE_DF.plot(x='FrameID', y='Horiz_Lip_Motion', style='o')

In [ ]:
OPEN_DF_0.plot(x='FrameID', y='Horiz_Lip_Motion', style='o')

In [ ]:
OPEN_DF_1.plot(x='FrameID', y='Horiz_Lip_Motion', style='o')

In [ ]:
OPEN_DF_0.plot(x='Time', y=['Vert_Lip_Motion','Horiz_Lip_Motion'] , style='o')
x = 2
plt.savefig('%d'%x)

In [ ]:
OPEN_DF_1.plot(x='Time', y='Vert_Lip_Motion', style='o')
x = 2
plt.savefig('%d'%x)

In [ ]:
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

for frame in range(OPEN_DF.shape[0]):
    ax.clear()
    data=OPEN_DF.iloc[[frame]]
    ax.scatter(data["X_LLeft"],data["Y_LLeft"], data["Z_LLeft"], color="r", label="Left Lip")
    ax.scatter(data["X_LR"],data["Y_LR"], data["Z_LR"], color="g", label="Right Lip")
    ax.scatter(data["X_UL"],data["Y_UL"], data["Z_UL"], color="b", label="Upper Lip")
    ax.scatter(data["X_LL"],data["Y_LL"], data["Z_LL"], color="b", label="Lower Lip")
    plt.savefig('fig_results/%d.png'%frame, bbox_inches='tight')

    

In [ ]:
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")


def connectpoints(DF_data, p1_sensor="",p2_sensor=""):
    # the choices are LLEFT, LR, LL, LU in string format
    x1 = data.iloc[0]["X_%s"%p1_sensor]
    y1 = data.iloc[0]["Y_%s"%p1_sensor]
    z1 = data.iloc[0]["Z_%s"%p1_sensor]

    x2 = data.iloc[0]["X_%s"%p2_sensor]
    y2 = data.iloc[0]["Y_%s"%p2_sensor]
    z2 = data.iloc[0]["Z_%s"%p2_sensor]

    plt.plot([x1,x2],[y1,y2],[z1,z2],'k-')



for frame in range(OPEN_DF.shape[0]):
    ax.clear()
    data=OPEN_DF.iloc[[frame]]
    ax.scatter(data["X_LLeft"],data["Y_LLeft"], data["Z_LLeft"], color="b", label="Left Lip")
    ax.scatter(data["X_LR"],data["Y_LR"], data["Z_LR"], color="g",label="Right Lip")
    ax.scatter(data["X_UL"],data["Y_UL"], data["Z_UL"], color="r",label="Upper Lip")
    ax.scatter(data["X_LL"],data["Y_LL"], data["Z_LL"], color="black",label="Lower Lip")
   

    connectpoints(data, "LLeft", "UL")
    connectpoints(data, "LLeft", "LL")
    connectpoints(data, p1_sensor="LR", p2_sensor="UL")
    connectpoints(data, p1_sensor="LR", p2_sensor="LL")
    #connectpoints(data, p1_sensor="UL", p2_sensor="LL")
    connectpoints(data, p1_sensor="LLeft", p2_sensor="LR")
    
    ax.set_xlim(-40, 40)
    ax.set_ylim(-90, -30)
    ax.set_zlim(-290, -200)
    
    ax.view_init(elev=-90, azim=-30)
    plt.savefig('fig_results_open/%d.png'%frame, bbox_inches='tight')

    

#connectpoints(x,y,z,2,3)




In [ ]:
%matplotlib notebook
%matplotlib notebook

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

def connectpoints(DF_data, p1_sensor="",p2_sensor=""):
    # the choices are LLEFT, LR, LL, LU in string format
    x1 = data.iloc[0]["X_%s"%p1_sensor]
    y1 = data.iloc[0]["Y_%s"%p1_sensor]
    z1 = data.iloc[0]["Z_%s"%p1_sensor]

    x2 = data.iloc[0]["X_%s"%p2_sensor]
    y2 = data.iloc[0]["Y_%s"%p2_sensor]
    z2 = data.iloc[0]["Z_%s"%p2_sensor]

    plt.plot([x1,x2],[y1,y2],[z1,z2],'k-')
    
data=OPEN_DF.iloc[[1000]]
ax.scatter(data["X_LLeft"],data["Y_LLeft"], data["Z_LLeft"], color="b", label="Left Lip")
ax.scatter(data["X_LR"],data["Y_LR"], data["Z_LR"], color="g",label="Right Lip")
ax.scatter(data["X_UL"],data["Y_UL"], data["Z_UL"], color="r",label="Upper Lip")
ax.scatter(data["X_LL"],data["Y_LL"], data["Z_LL"], color="black",label="Lower Lip")


connectpoints(data, "LLeft", "UL")
connectpoints(data, "LLeft", "LL")
connectpoints(data, p1_sensor="LR", p2_sensor="UL")
connectpoints(data, p1_sensor="LR", p2_sensor="LL")
#connectpoints(data, p1_sensor="UL", p2_sensor="LL")
connectpoints(data, p1_sensor="LLeft", p2_sensor="LR")

ax.set_xlim(-40, 40)
ax.set_ylim(-90, -30)
ax.set_zlim(-290, -200)
print (ax.azim)
ax.view_init(elev=-90, azim=-30)

In [ ]:
OPEN_DF[["Time", "X_LLeft","Y_LLeft","Z_LLeft", "X_LR","Y_LR","Z_LR","X_UL", "Y_UL", "Z_UL","X_LL","Y_LL","Z_LL"]].describe()




In [ ]:
%matplotlib notebook
%matplotlib notebook

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")


def connectpoints(DF_data, p1_sensor="",p2_sensor=""):
    # the choices are LLEFT, LR, LL, LU in string format
    x1 = data.iloc[0]["X_%s"%p1_sensor]
    y1 = data.iloc[0]["Y_%s"%p1_sensor]
    z1 = data.iloc[0]["Z_%s"%p1_sensor]

    x2 = data.iloc[0]["X_%s"%p2_sensor]
    y2 = data.iloc[0]["Y_%s"%p2_sensor]
    z2 = data.iloc[0]["Z_%s"%p2_sensor]

    plt.plot([x1,x2],[y1,y2],[z1,z2],'k-')




data=OPEN_DF.iloc[[frame]]
ax.scatter(data["X_LLeft"],data["Y_LLeft"], data["Z_LLeft"], color="b", label="Left Lip")
ax.scatter(data["X_LR"],data["Y_LR"], data["Z_LR"], color="g",label="Right Lip")
ax.scatter(data["X_UL"],data["Y_UL"], data["Z_UL"], color="r",label="Upper Lip")
ax.scatter(data["X_LL"],data["Y_LL"], data["Z_LL"], color="black",label="Lower Lip")


#connectpoints(data, "LLeft", "UL")
#connectpoints(data, "LLeft", "LL")
#connectpoints(data, p1_sensor="LR", p2_sensor="UL")
#connectpoints(data, p1_sensor="LR", p2_sensor="LL")
#connectpoints(data, p1_sensor="UL", p2_sensor="LL")
#connectpoints(data, p1_sensor="LLeft", p2_sensor="LR")
plt.show()

    

In [ ]:
nfr = len(OPEN_DF) # Number of frames
fps = 10 # Frame per sec

xs = []
ys = []
zs = []
ss = np.arange(1,nfr,1)
for s in ss:
    xs.append(np.array((OPEN_DF.iloc[0]["X_UL"], OPEN_DF.iloc[0]["X_LL"], OPEN_DF.iloc[0]["X_LLeft"],OPEN_DF.iloc[0]["X_LR"])))
    ys.append(np.array((OPEN_DF.iloc[0]["X_UL"], OPEN_DF.iloc[0]["X_LL"], OPEN_DF.iloc[0]["X_LLeft"],OPEN_DF.iloc[0]["X_LR"])))
    zs.append(np.array((OPEN_DF.iloc[0]["X_UL"], OPEN_DF.iloc[0]["X_LL"], OPEN_DF.iloc[0]["X_LLeft"],OPEN_DF.iloc[0]["X_LR"])))
   


In [ ]:
%matplotlib notebook
%matplotlib notebook
fig =  plt.figure()
ax = fig.add_subplot(111, projection='3d')
sct, = ax.plot([], [], [], "o", markersize=2)
def update(ifrm, xa, ya, za):
    sct.set_data(xa[ifrm], ya[ifrm])
    sct.set_3d_properties(za[ifrm])
ax.set_xlim(-100,100)
ax.set_ylim(-100,100)
ax.set_zlim(-100,100)
ani = animation.FuncAnimation(fig, update, nfr, fargs=(xs,ys,zs), interval=1000/fps,  repeat=False, blit=False)
